# Image Captioning Computer Vision.

Name: Anthony Kevin Oktavius

Paper and Dataset can be found at this link: https://paperswithcode.com/dataset/flickrstyle10k

Library Import

In [24]:
import numpy as np
import zipfile
import gc
import pickle
import cv2
import math
import warnings
import random
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import albumentations as A
import torchvision
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
import torch.utils.data as data_utils
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
# import torchmetrics
import torch.nn.functional as F
import PIL
import torch.utils.data as data_utils

# import pytorch_lightning as pl
import imutils
import zipfile

from torch.utils.data import Dataset
from collections import defaultdict
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.python.client import device_lib
from zipfile import ZipFile
from IPython import display
from torchvision import models, transforms
from google.colab.patches import cv2_imshow
from sklearn.metrics import confusion_matrix, roc_curve
# from pytorch_lightning.loggers import TensorBoardLogger

## Data Preparation

Download and Unzip the Dataset

In [40]:
! cp ./kaggle.json ~/.kaggle/
! kaggle datasets download -d adityajn105/flickr8k
! unzip -qq ./flickr8k.zip -d ./Dataset

flickr8k.zip: Skipping, found more recently modified local copy (use --force to force download)
replace ./Dataset/Images/1000268201_693b08cb0e.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


## Data Preprocessing

Create utility function to load datasets.

In [4]:
train_transform = transforms.Compose([
    transforms.CenterCrop((320, 320)),
    transforms.Resize((128, 128)), # Mitigate if there's a "RAM is not enough" error.
    transforms.RandomRotation(degrees = 5),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    # transforms.Normalize(
    #     mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225],
    # ),
])

val_transform = transforms.Compose([
    transforms.CenterCrop((224, 224)),
    transforms.Resize((128, 128)), # Mitigate if there's a "RAM is not enough" error.
    transforms.ToTensor(),
    # transforms.Normalize(
    #     mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225],
    # ),
])

Preprocess text

In [51]:
captions = open("Dataset/captions.txt", 'r')

image_name_to_labels_dict = defaultdict(list)
for text_per_line in captions.readlines()[1:]:
  image_name, label = text_per_line.split(".jpg,")
  image_name = f"{image_name}.jpg"
  label = label.strip("\n")

  image_name_to_labels_dict[image_name].append(label)

print("Example image_name to label dict: ")
print(image_name_to_labels_dict["1000268201_693b08cb0e.jpg"])
captions.close()

Example image_name to label dict: 
['A child in a pink dress is climbing up a set of stairs in an entry way .', 'A girl going into a wooden building .', 'A little girl climbing into a wooden playhouse .', 'A little girl climbing the stairs to her playhouse .', 'A little girl in a pink dress going into a wooden cabin .']


Divide into train and test folder.

In [ ]:
class FlickrStyleDataset(Dataset):
  def __init__(self, dataset_folder, dataset_labels, number_of_instances_per_class):
    super(FlickrStyleDataset, self).__init__()
    self.dataset_folder = dataset_folder
    self.dataset_labels = dataset_labels
    self.number_of_instances_per_class = number_of_instances_per_class

  def __getitem__(self):


  def __len__(self):
    return len(self.dataset_folder)

IndentationError: ignored

In [ ]:
TEST_SIZE = 0.2
TRAIN_SIZE = 1 - TEST_SIZE

random_split = torch.utils.data.random_split([tensor_tmnist_dataset,])

2513260012_03d33305cf.jpg
7000
7000


Generate a dataset loader.

## Data Modelling

In [ ]:
class Seq2SeqImageCaptioningModel(nn.Module):
  def __init__(self):
    self.

  def forward():
    pass